In [ ]:
# 正常bfs超时
from collections import deque
from typing import List


class Solution:

    def minReverseOperations(self, n: int, p: int, banned: List[int], k: int) -> List[int]:
        ans = [-1] * n
        vis = set(banned) | {p}
        q = deque([p])
        step = 0
        while q:
            for _ in range(len(q)):
                x = q.popleft()
                ans[x] = step
                for y in range(max(x - k + 1, k - 1 - x), min(x + k - 1, 2 * n - k - 1 - x) + 1, 2):
                    if y not in vis:
                        q.append(y)
                        vis.add(y)
            step += 1
        return ans

In [ ]:
# 利用并查集 删除（快速跳过）已经访问过的节点 （向右合并）
from bisect import bisect_left


class Solution:

    def minReverseOperations(self, n: int, p: int, banned: List[int], k: int) -> List[int]:
        ans = [-1] * n
        nb = [[], []]  # 偶数下标和奇数下标未访问
        vis = set(banned) | {p}
        for i in range(n):
            if i not in vis:
                nb[i & 1].append(i)
        nb[0].append(n)  # 哨兵
        nb[1].append(n)  # 哨兵
        roots = [list(range(len(nb[0]))), list(range(len(nb[1])))]  # 偶数下标和奇数下标的并查集

        def find(i, x):
            root = roots[i]
            if root[x] != x:
                root[x] = find(i, root[x])
            return root[x]

        def union(i, x, y):
            rx, ry = find(i, x), find(i, y)
            if rx != ry:
                roots[i][rx] = ry  # rx -> ry

        q = deque([p])
        step = 0
        while q:
            for _ in range(len(q)):
                x = q.popleft()
                ans[x] = step
                mn = max(x - k + 1, k - 1 - x)
                mx = min(x + k - 1, 2 * n - k - 1 - x)
                i = mn & 1
                j = find(i, bisect_left(nb[i], mn))  # >=mn最近没有ban的下标
                while nb[i][j] <= mx:
                    q.append(nb[i][j])
                    union(i, j, j + 1)
                    j = find(i, j + 1)
            step += 1
        return ans

In [ ]:
class Solution:

    def minReverseOperations(self, n: int, p: int, banned: List[int], k: int) -> List[int]:
        ans = [-1] * n
        roots = list(range(n + 2))

        def find(x):
            if roots[x] != x:
                roots[x] = find(roots[x])
            return roots[x]

        def union(x, y):
            rx, ry = find(x), find(y)
            if rx != ry:
                roots[rx] = ry  # rx -> ry

        for i in banned + [p]:
            union(i, i + 2)

        q = deque([p])
        step = 0
        while q:
            for _ in range(len(q)):
                x = q.popleft()
                ans[x] = step
                mn = max(x - k + 1, k - 1 - x)
                mx = min(x + k - 1, 2 * n - k - 1 - x)
                j = find(mn) 
                while j <= mx:
                    q.append(j)
                    union(j, j + 2)
                    j = find(j + 2)
            step += 1
        return ans